# Beef Export Network (Paraguay) - Edge and Node Construction

This notebook constructs the edge and node lists for the Paraguayan beef export network. 
The goal is to prepare structured inputs for network analysis by identifying co-export relationships between firms (edges) and summarizing exporter-level attributes (nodes).

### Section 1: Edge List - Co-exports by Year and Destination

In [58]:
import pandas as pd
from itertools import combinations

In [60]:
# Load beef export transaction dataset (Trase v1.2.4)
df = pd.read_csv("paraguay_beef_v1_2_4.csv")

In [62]:
# Select relevant columns
df_edges = df[['year', 'country_of_destination', 'exporter_group', 'fob', 'volume', 'land_use']].copy()

In [64]:
# Drop rows with missing key values
df_edges = df_edges.dropna(subset=['year', 'country_of_destination', 'exporter_group', 'fob'])

In [66]:
# Initialize list to store edges
edge_list = []

# Group by year and destination country
for (year, country), group in df_edges.groupby(['year', 'country_of_destination']):
    # Aggregate FOB by exporter
    exporters = group.groupby('exporter_group')['fob'].sum().reset_index()

    # Generate pairwise combinations of exporters
    for (exp1, fob1), (exp2, fob2) in combinations(exporters.values, 2):
        weight = fob1 + fob2  # Combined FOB value as edge weight
        edge_list.append({
            'source': exp1.strip().upper(),
            'target': exp2.strip().upper(),
            'year': year,
            'country': country,
            'weight': weight
        })

In [68]:
# Create edge DataFrame
edges_df = pd.DataFrame(edge_list)

In [70]:
# Export to CSV for network analysis
edges_df.to_csv("beef_coexport_edges.csv", index=False)

### Section 2: Node List - Aggregate exporter-level attributes across all years

In [73]:
# Standardize exporter names to ensure consistency
df['exporter_group'] = df['exporter_group'].str.strip().str.upper()

In [75]:
# Select relevant columns
df_nodes = df[['exporter_group', 'fob', 'volume', 'land_use']].copy()

In [77]:
# Drop rows with missing values
df_nodes = df_nodes.dropna(subset=['fob', 'volume', 'land_use'])

In [79]:
# Aggregate node-level attributes by exporter
nodes_df = (
    df_nodes
    .groupby('exporter_group')
    .agg(
        total_fob_value=('fob', 'sum'),
        total_volume=('volume', 'sum'),
        total_land_use=('land_use', 'sum')
    )
    .reset_index()
)

In [81]:
# Rename identifier column
nodes_df.rename(columns={'exporter_group': 'Id'}, inplace=True)

In [83]:
# Flag transnational corporations
transnational_firms = [
    'JBS',
    'MINERVA',
    'HAWKER INTERNATIONAL',
    'SWEET'
]

nodes_df['transnational'] = nodes_df['Id'].apply(lambda x: 'Oui' if x in transnational_firms else 'Non')

In [85]:
# Export node list to CSV
nodes_df.to_csv("beef_export_nodes.csv", index=False)